# Main Routine

This code is an updated Python adaption of Yvan Dossman's original Matlab code
by Callum Shaw



In [38]:
%matplotlib qt
#notebook backend important

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os.path
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askopenfilenames

In [ ]:

def index_find(depths,click):
    lst = depths>click
    for i,v in enumerate(lst):
        if v==True:
            return i
        

## Creating Density Profiles
Loading in the experiment density data from excel

In [ ]:
excel_path = askopenfilename() #path to excel doc

exp_num = int(input('Experiment number:'))
data = pd.read_excel(excel_path, skiprows=3) #has 3 unwanted rows at top

#getting data from excel doc
rho_bottom = data.iloc[exp_num, 10]
rho_top = data.iloc[exp_num, 11]
depth = data.iloc[exp_num, 7]


Loading in the background image and choosing region want to analyse (should be homogeneous). This image should contain no topography

In [4]:
background_path = askopenfilename() 
b_image = cv2.imread(background_path,0)

In [13]:

plt.figure()
plt.title('Background Image')
plt.imshow(b_image, cmap='gist_gray')
print('Select the left then the right portion of the region you want to average over')
crop_region = plt.ginput(2) #select the left then the right 
print('Locations recorded')

Select the left then the right portion of the region you want to average over
Locations recorded


Loading in the foreground images

In [14]:
#now loading in the foreground photos
foreground_path = askopenfilenames()
no_images = len(foreground_path)
os.mkdir('{}/results'.format(os.path.dirname(foreground_path[0]))) #making a directory to store results in same location as before


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'E:/Experiments/gamma/test_7/centre/results'

Now analysing the foreground photos

In [15]:
f_image = cv2.imread(foreground_path[0],0) #choosing the imag

f1=plt.figure()
plt.title('Foreground Image')
plt.imshow(f_image,  cmap='gist_gray') #plotting the f_image_crop check if happy 
print('Position of free surface then tank bottom')
free_surface = plt.ginput(2)
print('Locations recorded')

f2=plt.figure()
f_image_crop = f_image[int(free_surface[0][1]):int(free_surface[1][1]),int(crop_region[0][0]):int(crop_region[1][0])]
plt.title('Foreground Image Cropped')
plt.imshow(f_image_crop,  cmap='gist_gray') #plotting the f_image_crop check if happy 

#cropping the background image the same way
b_image_crop = b_image[int(free_surface[0][1]):int(free_surface[1][1]),int(crop_region[0][0]):int(crop_region[1][0])]
b_image_crop = b_image_crop*0+1 #in the case of an initially varing stratification

Position of free surface then tank bottom
Locations recorded


Absorbtion Calculation

In [16]:
ratio =f_image_crop/b_image_crop
log_ratio = np.log(ratio)

#getting rid of unwated inf_values
log_ratio[np.isinf(log_ratio)]=np.nan

mean_ratio_profile = np.mean(log_ratio,1)
intensity = mean_ratio_profile[::-1]

depth_array = np.linspace(0,depth,len(mean_ratio_profile))
gamma=depth/(int(free_surface[1][1])-int(free_surface[0][1]))
plt.figure()
plt.plot(intensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Raw Absorption Profile')

Text(0.5, 1.0, 'Raw Absorption Profile')

Spline of the intensity profile- fixing the top and bottom. Using constant at top and linear slope at bottom

In [17]:
#firstly looking at the top section 
splintensity=np.copy(intensity)

plt.figure()
plt.plot(intensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Top Section of Absorption Profile')
print('Choose bottom of top section')
topsection = plt.ginput(1)[0][1]
print('Location recorded')
ztop=index_find(depth_array,topsection) 
splintensity[ztop:]=splintensity[ztop]

Choose bottom of top section
Location recorded


In [18]:
#now looking at the bottom section 
plt.figure()
plt.plot(splintensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Bottom Section of Absorption Profile')
print('Choose top then bottom')
bottomsection = plt.ginput(2)
print('Locations recorded')

Choose top then bottom
Locations recorded


In [19]:
slope = (bottomsection[0][0]-bottomsection[1][0])/bottomsection[0][1]
intercept = bottomsection[0][1]-bottomsection[0][0]/slope
zbot=index_find(depth_array, bottomsection[0][1]) 
splintensity[0:zbot]=slope*(np.linspace(0,bottomsection[0][1], zbot)-intercept)
plt.figure()
plt.title('Absorbtion Profile')
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.plot(intensity,depth_array, 'b', label='Raw')
plt.plot(splintensity,depth_array, 'r',label='Updataed')
plt.legend()


Check that plot above looks good or repeat as needed

## Now converting from intensity to density


In [20]:
#determining the location of density samples (rho_bottom, rho_top)

plt.figure()
plt.title('Density Sample Locations')
plt.ylabel('z (m)')
plt.imshow(f_image_crop, cmap='gist_gray')
print('Position of density sample close to bottom, then close to free surface' )
density_locations = plt.ginput(2)
print('Locations recorded')

Position of density sample close to bottom, then close to free surface
Locations recorded


In [21]:
zbot_rho=int(np.round(density_locations[0][1]))
ztop_rho=int(np.round(density_locations[1][1]))

beta = (rho_bottom-rho_top)/(splintensity[zbot_rho]-splintensity[ztop_rho])
splintensity_bottom_ref = splintensity[zbot_rho]

rho = rho_bottom+beta*(splintensity-splintensity_bottom_ref)

In [35]:
plt.figure()
plt.title('Density Profile')
plt.xlabel('Density (kg/m3)')
plt.ylabel('z (m)')
plt.plot(rho,depth_array, 'b', label='Raw')



In [ ]:
#do not currently have mass corention included

In [23]:
np.savez('{}/results/profile_0'.format(os.path.dirname(foreground_path[0])),gamma,beta,splintensity,rho,rho_bottom,rho_top,depth_array)

In [ ]:
#running over-remaining pictures
f_image = cv2.imread(foreground_path[0],0)

In [39]:
for i in range(no_images-1):
    p = i+1
    
    #reading in image
    image = cv2.imread(foreground_path[p],0)
    image_crop=image[int(free_surface[0][1]):int(free_surface[1][1]),int(crop_region[0][0]):int(crop_region[1][0])]
    
    #creating profile
    ratio =image_crop/b_image_crop
    log_ratio = np.log(ratio)

    #getting rid of unwated inf_values
    log_ratio[np.isinf(log_ratio)]=np.nan
    mean_ratio_profile = np.mean(log_ratio,1)
    intensity = mean_ratio_profile[::-1]
    
    splintensity=np.copy(intensity)
    splintensity[ztop:]=splintensity[ztop]
    splintensity[0:zbot]=slope*(np.linspace(0,bottomsection[0][1], zbot)-intercept)
    
    beta = (rho_bottom-rho_top)/(splintensity[zbot_rho]-splintensity[ztop_rho])
    splintensity_bottom_ref = splintensity[zbot_rho]
    rho= rho_bottom+beta*(splintensity-splintensity_bottom_ref)
    np.savez('{}/results/profile_{}'.format(os.path.dirname(foreground_path[0]),p),gamma,beta,splintensity,rho,rho_bottom,rho_top,depth_array)
    plt.title('Density Profile')
    plt.xlabel('Density (kg/m3)')
    plt.ylabel('z (m)')
    plt.plot(rho,depth_array)
